# Feature Extract

In [206]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from utils import *

## Train Set

In [51]:
# Get the data:
train_filepath = 'data/train.csv'
traindf = pd.read_table(train_filepath)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [52]:
# Inspect the schema
traindf.columns

Index(['Row', 'Anon Student Id', 'Problem Hierarchy', 'Problem Name',
       'Problem View', 'Step Name', 'Step Start Time',
       'First Transaction Time', 'Correct Transaction Time', 'Step End Time',
       'Step Duration (sec)', 'Correct Step Duration (sec)',
       'Error Step Duration (sec)', 'Correct First Attempt', 'Incorrects',
       'Hints', 'Corrects', 'KC(Default)', 'Opportunity(Default)'],
      dtype='object')

In [53]:
# Inspect the head of data
traindf.head(10)

,Row,Anon Student Id,Problem Hierarchy,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,Step End Time,Step Duration (sec),Correct Step Duration (sec),Error Step Duration (sec),Correct First Attempt,Incorrects,Hints,Corrects,KC(Default),Opportunity(Default)
0,9938,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R1C1,2005-09-09 12:23:34.0,2005-09-09 12:24:07.0,2005-09-09 12:24:07.0,2005-09-09 12:24:07.0,33.0,33.0,NaN,1,0,0,1,NaN,NaN
1,9939,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R1C2,2005-09-09 12:24:07.0,2005-09-09 12:24:22.0,2005-09-09 12:24:22.0,2005-09-09 12:24:22.0,15.0,15.0,NaN,1,0,0,1,NaN,NaN
2,9940,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R3C1,2005-09-09 12:24:22.0,2005-09-09 12:25:16.0,2005-09-09 12:25:40.0,2005-09-09 12:25:40.0,78.0,NaN,78.0,0,2,0,1,Define Variable,1
3,9941,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R3C2,2005-09-09 12:25:40.0,2005-09-09 12:25:51.0,2005-09-09 12:27:24.0,2005-09-09 12:27:24.0,104.0,NaN,104.0,0,4,9,1,"Using small numbers~~Write expression, positiv...",1~~1~~1
4,9942,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R4C1,2005-09-09 12:27:24.0,2005-09-09 12:27:30.0,2005-09-09 12:27:30.0,2005-09-09 12:27:30.0,6.0,6.0,NaN,1,0,0,1,Entering a given,1
5,9943,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R5C1,2005-09-09 12:27:30.0,2005-09-09 12:27:41.0,2005-09-09 12:27:41.0,2005-09-09 12:27:41.0,11.0,11.0,NaN,1,0,0,1,Entering a given,2
6,9944,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R6C1,2005-09-09 12:27:41.0,2005-09-09 12:28:41.0,2005-09-09 12:28:41.0,2005-09-09 12:28:41.0,60.0,60.0,NaN,1,0,0,1,Entering a given,3
7,9945,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R7C2,2005-09-09 12:28:50.0,2005-09-09 12:28:58.0,2005-09-09 12:28:58.0,2005-09-09 12:28:58.0,8.0,8.0,NaN,1,0,0,1,Entering a given,4
8,9946,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,ValidEquations,2005-09-09 12:31:39.0,2005-09-09 12:31:52.0,2005-09-09 12:31:52.0,2005-09-09 12:31:52.0,13.0,13.0,NaN,1,0,0,2,NaN,NaN
9,9947,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,7/10*X = 1400,2005-09-09 12:31:52.0,2005-09-09 12:32:28.0,2005-09-09 12:32:35.0,2005-09-09 12:32:35.0,43.0,NaN,43.0,0,1,0,1,NaN,NaN


In [54]:
# Apply df as alias for training data in the following computation
df = traindf
df.dtypes

Row                              int64
Anon Student Id                 object
Problem Hierarchy               object
Problem Name                    object
Problem View                     int64
Step Name                       object
Step Start Time                 object
First Transaction Time          object
Correct Transaction Time        object
Step End Time                   object
Step Duration (sec)            float64
Correct Step Duration (sec)    float64
Error Step Duration (sec)      float64
Correct First Attempt            int64
Incorrects                       int64
Hints                            int64
Corrects                         int64
KC(Default)                     object
Opportunity(Default)            object
dtype: object

In [210]:
# Clean up in dataframe, to remove redundant columns
# maybe some columns are deprecated, and no more used,
# fill them out of the following list and run this cell
# to do cleanup
columns_to_remove = ["CFA | Problem Name,Step Name"]
df = df.drop(columns=columns_to_remove)
df.columns

Index(['Row', 'Anon Student Id', 'Problem Hierarchy', 'Problem Name',
       'Problem View', 'Step Name', 'Step Start Time',
       'First Transaction Time', 'Correct Transaction Time', 'Step End Time',
       'Step Duration (sec)', 'Correct Step Duration (sec)',
       'Error Step Duration (sec)', 'Correct First Attempt', 'Incorrects',
       'Hints', 'Corrects', 'KC(Default)', 'Opportunity(Default)',
       'CFA | Anon Student Id', 'CFA | Step Name', 'CFA | Problem Name',
       'CFA | KC(Default)', 'CFA | Anon Student Id,Problem Name',
       'CFA | Anon Student Id,KC(Default)'],
      dtype='object')

In [90]:
def CFAR(df, idx, columns, newcol, itercount, totalcount):
    if len(columns) == 0:
        cfa = df.loc[idx & (df["Correct First Attempt"] == 1)]
        df.loc[idx, newcol] = len(cfa) / len(df.loc[idx]) if len(df.loc[idx]) != 0 else 0.0
        
        # Update progress
        print("%.1f%%" % (itercount / totalcount * 100.0), end='\r')
        itercount+=1
    else:
        col = columns[0]
        keys = set(df[col])
        print("%s: %d" % (col, len(keys)))
        for k in keys:
            if idx is None:
                local_idx = (df[col] == k)
            else:
                local_idx = idx & (df[col] == k)
            CFAR(df, local_idx, columns[1:], newcol)
            
def computeCFAR(df, columns):
    newcol = nameOfCFAR(columns)
    df[newcol] = pd.Series(np.zeros(len(df)), index=df.index)
    total = 1
    for col in columns:
        total *= len(set(df[col]))
    CFAR(df, None, columns, newcol, 0, total)

In [91]:
# CFA | Student Name
computeCFAR(df, ["Anon Student Id"])

In [95]:
# CFA | Step Name
computeCFAR(df, ["Step Name"])

In [92]:
# CFA | Problem Name
computeCFAR(df, ["Problem Name"])

In [93]:
# CFA | KC
computeCFAR(df, ["KC(Default)"])

In [ ]:
# CFA | Problem Name, Step Name
computeCFAR(df, ["Problem Name", "Step Name"])

In [156]:
# CFA | Student Name, Problem Name
computeCFAR(df, ["Anon Student Id", "Problem Name"])

In [94]:
# CFA | Student Name, KC
computeCFAR(df, ["Anon Student Id", "KC(Default)"])

In [211]:
# Inspect the head of new features
df.head(100)

,Row,Anon Student Id,Problem Hierarchy,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,Step End Time,...,Hints,Corrects,KC(Default),Opportunity(Default),CFA | Anon Student Id,CFA | Step Name,CFA | Problem Name,CFA | KC(Default),"CFA | Anon Student Id,Problem Name","CFA | Anon Student Id,KC(Default)"
0,9938,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R1C1,2005-09-09 12:23:34.0,2005-09-09 12:24:07.0,2005-09-09 12:24:07.0,2005-09-09 12:24:07.0,...,0,1,NaN,NaN,0.748749,0.840631,0.710197,0.000000,0.666667,0.000000
1,9939,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R1C2,2005-09-09 12:24:07.0,2005-09-09 12:24:22.0,2005-09-09 12:24:22.0,2005-09-09 12:24:22.0,...,0,1,NaN,NaN,0.748749,0.830699,0.710197,0.000000,0.666667,0.000000
2,9940,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R3C1,2005-09-09 12:24:22.0,2005-09-09 12:25:16.0,2005-09-09 12:25:40.0,2005-09-09 12:25:40.0,...,0,1,Define Variable,1,0.748749,0.966979,0.710197,0.966979,0.666667,0.965517
3,9941,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R3C2,2005-09-09 12:25:40.0,2005-09-09 12:25:51.0,2005-09-09 12:27:24.0,2005-09-09 12:27:24.0,...,9,1,"Using small numbers~~Write expression, positiv...",1~~1~~1,0.748749,0.404477,0.710197,0.443541,0.666667,0.413793
4,9942,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R4C1,2005-09-09 12:27:24.0,2005-09-09 12:27:30.0,2005-09-09 12:27:30.0,2005-09-09 12:27:30.0,...,0,1,Entering a given,1,0.748749,0.760107,0.710197,0.817953,0.666667,0.815287
5,9943,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R5C1,2005-09-09 12:27:30.0,2005-09-09 12:27:41.0,2005-09-09 12:27:41.0,2005-09-09 12:27:41.0,...,0,1,Entering a given,2,0.748749,0.755559,0.710197,0.817953,0.666667,0.815287
6,9944,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R6C1,2005-09-09 12:27:41.0,2005-09-09 12:28:41.0,2005-09-09 12:28:41.0,2005-09-09 12:28:41.0,...,0,1,Entering a given,3,0.748749,0.694107,0.710197,0.817953,0.666667,0.815287
7,9945,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R7C2,2005-09-09 12:28:50.0,2005-09-09 12:28:58.0,2005-09-09 12:28:58.0,2005-09-09 12:28:58.0,...,0,1,Entering a given,4,0.748749,0.757615,0.710197,0.817953,0.666667,0.815287
8,9946,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,ValidEquations,2005-09-09 12:31:39.0,2005-09-09 12:31:52.0,2005-09-09 12:31:52.0,2005-09-09 12:31:52.0,...,0,2,NaN,NaN,0.748749,0.880437,0.710197,0.000000,0.666667,0.000000
9,9947,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,7/10*X = 1400,2005-09-09 12:31:52.0,2005-09-09 12:32:28.0,2005-09-09 12:32:35.0,2005-09-09 12:32:35.0,...,0,1,NaN,NaN,0.748749,0.571429,0.710197,0.000000,0.666667,0.000000


In [213]:
# Save these precious features into disk !!
df.to_csv("data/CFAR_features_train.csv", sep='\t')

## Test Set

In [100]:
# Get the test data:
test_filepath = 'data/test.csv'
testdf = pd.read_table(test_filepath)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [101]:
# Inspect schema
testdf.columns
# Inspect head of testdf
testdf.head(20)

,Row,Anon Student Id,Problem Hierarchy,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,Step End Time,Step Duration (sec),Correct Step Duration (sec),Error Step Duration (sec),Correct First Attempt,Incorrects,Hints,Corrects,KC(Default),Opportunity(Default)
0,10039,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP01,1,2/300*X = 10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10482,52vEY7f17k,"Unit CTA1_06, Section CTA1_06-3",FOR05,1,xScale,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Changing axis intervals,28
2,11024,52vEY7f17k,"Unit ES_03, Section ES_03-6",EG52,1,2.9 = -2x+3.7+x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[SkillRule: Consolidate vars, no coeff; CLT]",27
3,11476,52vEY7f17k,"Unit CTA1_08, Section CTA1_08-3",REAL10,1,yScale,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Changing axis intervals,65
4,11948,52vEY7f17k,"Unit ES_04, Section ES_04-12",EG58,7,xR2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,12532,52vEY7f17k,"Unit CTA1_10, Section CTA1_10-5",DIST05_SP,1,R1C2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,12955,52vEY7f17k,"Unit ES_07, Section ES_07-4",LIT69A,4,z*u-n*j+z*g = 16n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[SkillRule: Consolidate vars, any; {Combine va...",42
7,19200,6W08a98ZQV,"Unit ES_02, Section ES_02-8",EG41,2,FinalAnswer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,perform-mult-sp,9
8,19384,6W08a98ZQV,"Unit CTA1_13, Section CTA1_13-1",PROP10,1,R1C2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,19835,6W08a98ZQV,"Unit CTA1_06, Section CTA1_06-3",FOR05,2,XLabel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Labelling the axes,33


In [127]:
# Query CFAR through the specified columns as keys
def queryCFAR(row, df, columns):
    idx = None
    CFAR = 0.0
    for col in columns:
        if idx is None:
            idx = (df[col] == row[col])
        else:
            idx &= (df[col] == row[col])
    sub = df.loc[idx]
    if len(sub) != 0:
        CFAR = sub.iloc[0].loc[nameOfCFAR(columns)]
    return CFAR

In [129]:
CFAR_features = [ ["Anon Student Id"], ["Step Name"], ["Problem Name"], ["KC(Default)"]
    , ["Anon Student Id", "Problem Name"], ["Anon Student Id", "KC(Default)"]
]
for f in CFAR_features:
    testdf[nameOfCFAR(f)] = testdf.apply(queryCFAR, axis='columns', args=(traindf, f))

In [130]:
# Inspect the head of new features
testdf.head(100)

,Row,Anon Student Id,Problem Hierarchy,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,Step End Time,...,Hints,Corrects,KC(Default),Opportunity(Default),CFA | Anon Student Id,CFA | Step Name,CFA | Problem Name,CFA | KC(Default),"CFA | Anon Student Id,Problem Name","CFA | Anon Student Id,KC(Default)"
0,10039,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP01,1,2/300*X = 10,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.748749,0.800000,0.858006,0.000000,1.000000,0.000000
1,10482,52vEY7f17k,"Unit CTA1_06, Section CTA1_06-3",FOR05,1,xScale,NaN,NaN,NaN,NaN,...,NaN,NaN,Changing axis intervals,28,0.748749,1.000000,0.766129,1.000000,0.666667,1.000000
2,11024,52vEY7f17k,"Unit ES_03, Section ES_03-6",EG52,1,2.9 = -2x+3.7+x,NaN,NaN,NaN,NaN,...,NaN,NaN,"[SkillRule: Consolidate vars, no coeff; CLT]",27,0.748749,0.000000,0.877049,0.895683,1.000000,0.790698
3,11476,52vEY7f17k,"Unit CTA1_08, Section CTA1_08-3",REAL10,1,yScale,NaN,NaN,NaN,NaN,...,NaN,NaN,Changing axis intervals,65,0.748749,1.000000,0.812217,1.000000,1.000000,1.000000
4,11948,52vEY7f17k,"Unit ES_04, Section ES_04-12",EG58,7,xR2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.748749,0.666041,0.869446,0.000000,0.805556,0.000000
5,12532,52vEY7f17k,"Unit CTA1_10, Section CTA1_10-5",DIST05_SP,1,R1C2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.748749,0.830699,0.807692,0.000000,0.000000,0.000000
6,12955,52vEY7f17k,"Unit ES_07, Section ES_07-4",LIT69A,4,z*u-n*j+z*g = 16n,NaN,NaN,NaN,NaN,...,NaN,NaN,"[SkillRule: Consolidate vars, any; {Combine va...",42,0.748749,0.250000,0.727554,0.566392,0.750000,0.500000
7,19200,6W08a98ZQV,"Unit ES_02, Section ES_02-8",EG41,2,FinalAnswer,NaN,NaN,NaN,NaN,...,NaN,NaN,perform-mult-sp,9,0.777308,0.649245,0.793210,0.867826,0.709091,0.000000
8,19384,6W08a98ZQV,"Unit CTA1_13, Section CTA1_13-1",PROP10,1,R1C2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.777308,0.830699,0.844037,0.000000,1.000000,0.000000
9,19835,6W08a98ZQV,"Unit CTA1_06, Section CTA1_06-3",FOR05,2,XLabel,NaN,NaN,NaN,NaN,...,NaN,NaN,Labelling the axes,33,0.777308,0.894722,0.766129,0.859338,0.625000,0.802817


In [214]:
# Save these precious features into disk !!
testdf.to_csv("data/CFAR_features_test.csv", sep='\t')